# LDA Topic Prediction on Song Tags

This notebook will take the tags belonging to our song dataset and turn them into vectors of topics using latent Dirichlet allocation (LDA). A deep dive on this topic is available at the Sagemaker documentation here: https://docs.aws.amazon.com/sagemaker/latest/dg/lda.html

Credit is due to the AWS documentation for some blocks of code setting up the training and preparation of this data.

Our first step, as always, will be to bring in the packages we will need.

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import json
import itertools as it
import json
import datetime as dt
import string
import boto3
from decimal import Decimal

import sagemaker
from sagemaker.amazon.common import numpy_to_record_serializer
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac
import io

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Data Preparation

A lot of the data cleaning has taken place in the EC2 instance in which the data was retrieved. There is little left for us to do in this notebook but convert the cleaned tags into a machine readable format for training.

Below, we instantiate a TFIDF vectorizer class that we will use to convert the tags into floating points.

In [6]:
#we're going to use a TFIDF to encode the tags for each song
#setting the minimum number of times a word occurs to 50 to get counted
tf = TfidfVectorizer(
    stop_words='english',
    sublinear_tf=True,
    strip_accents='unicode',
#     max_features= 25000
    min_df = 50
)

In [7]:
#pointing to the data we'll use for training
bucket='sagemaker-msdsubset'
data_key = 'flat_summary_04_09_20.csv'
prefix = 'sagemaker/preprocessed_lda'
role = get_execution_role()

data_location = 's3://{}/{}'.format(bucket, data_key)

flat = pd.read_csv(data_location, names = ['song_id', 'track_id', 'song_hotness', 'artist_familiarity',
       '7digital_id', 'title', 'artist', 'mode', 'tempo', 'key', 'artist_id',
       'spotify_uri_final', 'last_fm_tags'])

#the LDA can't predict on empty strings, so we have to drop those songs
flat = flat[~flat['last_fm_tags'].isnull()]

flat.head()

,song_id,track_id,song_hotness,artist_familiarity,7digital_id,title,artist,mode,tempo,key,artist_id,spotify_uri_final,last_fm_tags
0,SOQMMHC12AB0180CB8,TRMMMYQ128F932D901,0.542899,0.649822,7032331,Silent Night,Faster Pussy cat,0,87.002,10,ARYZTJS1187B98C555,NaN,"heavymetal,industrialmetal,hardrock,glammetal,..."
1,SOVFVAK12A8C1350D9,TRMMMKD128F425225D,0.299877,0.439604,1514808,Tanssi vaan,Karkkiautomaatti,1,150.778,9,ARMVN3U1187FB3A1EB,spotify:track:6DOmOjeTc3btomrfFfPgy8,"poprock,indierock,chillout,rock,alternativeroc..."
2,SOGTUKN12AB017F4F1,TRMMMRX128F93187D9,0.617871,0.643681,6945353,No One Could Ever,Hudson Mohawke,1,177.768,7,ARGEKB01187FB50750,spotify:track:41RpZW2lxAdnqDd2nMBzLQ,"brokenbeat,hiphop,triphop,glitch,ghettotech,ro..."
3,SOBNYVR12A8C13558C,TRMMMCH128F425532C,0.000000,0.448501,2168257,Si Vos Querés,Yerba Brava,1,87.433,7,ARNWYLR1187B9B2F9C,spotify:track:7z4BZV7eZO1bqVKwAeTmou,"cumbia,italiandisco,losangeles,electronic,coun..."
4,SOHSBXH12A8C13B0DF,TRMMMWA128F426B589,0.000000,0.000000,2264873,Tangle Of Aspens,Der Mystic,0,140.035,5,AREQDTE1269FB37231,spotify:track:2poHURuOfVNbzZdivAwtOH,"hardtrance,darkpop,trance,electronica,dub,elec..."


In [9]:
#we're going to use TFIDF on the tags
##this was instantiated above
x_tf = tf.fit_transform(flat['last_fm_tags'])

#put them into a bytesIO object
#this is because TFIDF returns a sparse matrix
##and memory isnt sufficient for dense 
buf = io.BytesIO()
smac.write_spmatrix_to_sparse_tensor(file = buf, array = x_tf)
buf.seek(0)

#Sagemaker trains from data stored in an S3 bucket
#so let's upload to this S3 bucket
train_key = 'recordio-tfidf-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', train_key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, train_key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://sagemaker-msdsubset/sagemaker/preprocessed_lda/train/recordio-tfidf-data


# Training

Below, we read in the most recent image of our training algorithm. We then set up the hyperparameters of the training instance and fit it on the S3 instance we uploaded above.

In [10]:
from sagemaker.amazon.amazon_estimator import get_image_uri
# select the algorithm container based on this notebook's current location

region_name = boto3.Session().region_name
container = get_image_uri(region_name, 'lda')

In [11]:
session = sagemaker.Session()

# specify general training job information
lda = sagemaker.estimator.Estimator(
    container,
    role,   #environment variable set above
    output_path='s3://{}/{}/output'.format(bucket, prefix), #point to our data
    train_instance_count=1, #LDA only supports single CPU training
    train_instance_type='ml.c4.2xlarge',
    sagemaker_session=session,
)

# set algorithm-specific hyperparameters
lda.set_hyperparameters(
    num_topics=10, #selected due to sparsity of tags in dataset for each song
    feature_dim=len(tf.vocabulary_),
    mini_batch_size=x_tf.shape[0],
    alpha0=1.0,
)

# run the training job on input data stored in S3
lda.fit({'train': s3_train_data})

2020-04-11 16:29:25 Starting - Starting the training job...
2020-04-11 16:29:26 Starting - Launching requested ML instances...
2020-04-11 16:30:21 Starting - Preparing the instances for training......
2020-04-11 16:31:24 Downloading - Downloading input data
2020-04-11 16:31:24 Training - Downloading the training image......
2020-04-11 16:32:11 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Using mxnet backend.
[04/11/2020 16:32:13 INFO 139870123513664] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'alpha0': u'1.0', u'max_restarts': u'10', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'allow_svd_init': u'true', u'epochs': u'1', u'tol': u'1e-8', u'_kvstore': u'local', u'max_iterations': u'1000'}
[04/11/2020 16:32:13 INFO 139870123513664] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'2928

[04/11/2020 16:32:35 INFO 139870123513664] Krylov method iteration: 4.
[04/11/2020 16:32:38 INFO 139870123513664] Krylov method iteration: 5.
[04/11/2020 16:32:41 INFO 139870123513664] Krylov method iteration: 6.
[04/11/2020 16:32:47 INFO 139870123513664] Covariance matrix min value: -0.000002
[04/11/2020 16:32:47 INFO 139870123513664] Covariance matrix max value: 0.000018
[04/11/2020 16:32:47 INFO 139870123513664] Starting SVD...
#metrics {"Metrics": {"svd.time": {"count": 1, "max": 10.989904403686523, "sum": 10.989904403686523, "min": 10.989904403686523}}, "EndTime": 1586622767.263636, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "LDA"}, "StartTime": 1586622744.439496}

[04/11/2020 16:32:47 INFO 139870123513664] Finished SVD.
#metrics {"Metrics": {"rand_svd.time": {"count": 1, "max": 22826.735019683838, "sum": 22826.735019683838, "min": 22826.735019683838}}, "EndTime": 1586622767.266264, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": 


2020-04-11 16:33:03 Uploading - Uploading generated training model
2020-04-11 16:33:03 Completed - Training job completed
Training seconds: 116
Billable seconds: 116


# Endpoint Deployment and Prediction

Below, we deploy the endpoint so that we can make predictions and generate the topic vectors for our data. We iterate through all million of our songs and generate 10 topic vectors for all of them.

This part takes approximately 30 hours.

In [12]:
#deploy our endpoint for inference belo
lda_inference = lda.deploy(
    initial_instance_count=1,
    instance_type='ml.c4.xlarge',
)
lda_inference.content_type = 'text/csv'
lda_inference.serializer = csv_serializer
lda_inference.deserializer = json_deserializer

-------------!

In [ ]:
empty = pd.DataFrame()
#Sagemaker only supports 10 predictions out of the endpoint at a time
#so we'll have to spoonfeed ten at a time and put the results in "empty"
inc = 10
for i in range(0,len(flat),inc): 
    
    #make the prediction
    if i+10 > len(flat):
        test = lda_inference.predict(np.array(x_tf[i:,:].todense()))
    else:
        test = lda_inference.predict(np.array(x_tf[i:i+inc,:].todense()))
    
    #turn it into a DF
    to_s3 = pd.DataFrame(pd.DataFrame.from_dict(test['predictions'])['topic_mixture'].values.tolist())
    
    #put the song IDs back on the predicted DF
    if i+inc > len(flat):
        to_s3 = pd.concat([to_s3,flat['song_id'].reset_index(drop=True)[i:].reset_index(drop=True)],axis = 1)

    else:
        to_s3 = pd.concat([to_s3,flat['song_id'].reset_index(drop=True)[i:i+inc].reset_index(drop=True)],axis = 1)
    
    #and append to empty
    empty = empty.append(to_s3)

    #a small progress report
    if i % 50000 == 0:
        print(i)

#this will eventually be uploaded to RDS, which can't accept dupes
##so enforce it at this point
empty.drop_duplicates('song_id', inplace = True)

#again, will be uploaded to RDS, which doesn't accept header or index
empty.to_csv('inference.csv', index=False, header=False)
csv_object = os.path.join(prefix, 'inference', 'inference.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(csv_object).upload_file('inference.csv')

850000
900000
